In [ ]:
#Jonathan Borowsky
#1/29/25
#sampling methods

In [ ]:
#contain objects needed to extend each sampling method
class sampler_state():
    pass

# generate initial coordinates and objects like metadynamics grids for the sampling method
def sampling_method_init(sampler_state, etc):
    return sampler_state

# run a sampling method for which constants, input objects like metadynamics grids, and initial conditions are already set up
def sampling_method_extension(sampler_state, etc):
    return sampler_state

def energy_mfpt_estimate(sampler_state):
    return energy_est, mfpt_est


#new outline
#parameters
# system: the PhD student's system
#      the system contains either a potential energy function or a markov state model, and in either case generates a trajectory x(t) and a state trajectory s(t), 
#      which is passed around along with x(t) by the sampling method only used inside the system method when dealing with msm-based systems
#      each system consists of a class describing a particular energy landscape and a superclass with methods for which different versions are needed for msm- and non-msm- based systems. 
#      However all of these methods should have the same names, external arguments, and return types so that the sampling methods can call them with no regard for what kind of system is being used
# max_molecular_time: how long the PhD student has until they need to graduate
# max_aggregate_time: how much money the PI has to spend on compute or how large of a cluster the university has
# featurization_binning_scheme: algorithm of compressing the system's trajectory into a lower-dimensional and/or discrete representation
#     this method is composed of a featurization scheme to compress the data into a lower dimensional space and a binning scheme to discretize it. 
#     It returns both the compressed and discretized data even though the sampling method may not use both (i.e. gridless metadynamics)
#     or the featurization scheme may do nothing (i.e. for a 1d system where no dimensionality reduction is called for)
#     this could be broken into two functions and probably should.
#     this argument is independent from the system because the system could be analyzed with both different methods (mtd, we) and different dimensionality reduction (pca, tica) and different discretization (kmeans, kmedoids)
#     this argument is independent from the sampling method because the binning and dimensionality reduction scheme is often informed by biological knowledge of the specific system
#        however, this breaks down if the sampling method depends on specific binning or dimensionality reduction or uses multiple kinds thereof.
#        in most MD projects feature selection precedes adaptive/enhanced sampling but varies by system
#     replace this with a tuple of auxilliary modular methods that the sampling method may use, to be generated by itertools in the bootstrapping method
# n_timepoints: number of timepoints to measure convergence at; has no effect on the sampling method

def sampling_method(system, max_molecular_time, max_aggregate_time, featurization_binning_scheme, n_timepoints):
    #calculate number of parallel walkers using aggregate and molecular time if necessary

    #initialize starting and constant values for sampling method
    sampler_state = sampling_method_init(args above) #make this an object if necessary, containing things like coordinates, weights, and the metadynamics grid

    energy_mfpt_ests = []

    #loop to save out data at intervals
    for nt in range(t_timepoints):
        sampler_state = sampling_method_extension(sampler_state, etc)
        energy_mfpt_ests.append(energy_mfpt_estimate(sampler_state))
    
    return energy_mfpt_ests

#we need a generalized plotting method that can plot 1- and 2- dimensional slices of 3d data.




In [ ]:
class _data():

    def __init__():
        pass

    method_info = [] #which combination of methods was used to generate this data
    landscape_estimates = [[],...] #incl. coordinates, length = n_timepoints
    mfpt_estimates = [[],...]

    def add_replicate(self, ...):
        pass

    def calculate_error_bars_convergence():
        pass
        #average across replicates and deal with once-sampled states, and calculate convergence over n_timepoints


#sampling_methods and aux_functions could be combined into a single list of lists

def bootstrapping_method(systems, sampling_methods, aux_functions, max_molecular_time, max_aggregate_time, n_timepoints, n_replicates):
    #aux_function_combos = itertools.product(aux_functions)
    #i.e. if aux_functions = [[dimensionality_reduction_1, dimensionality_reduction_2],[binning_1, binning_2]]
    #    then aux_function_combos = [[dimensionality_reduction_1, binning_1],[dimensionality_reduction_2, binning_1],[dimensionality_reduction_1, binning_2],[dimensionality_reduction_2, binning_2]]

    all_scanned_params = itertools.product(systems+sampling_methods+aux_functions)

    paramscan_bootstrap_data = []

    for combo in all_scanned_params:
        data1 = _data(combo)
        for nr in n_replicates:
            data1.add_replicate(combo[0](combo, max_molecular_time, max_aggregate_time, n_timepoints))

        data1.calculate_error_bars_convergence
        paramscan_bootstrap_data.append(data1)

    return paramscan_bootstrap_data


    


In [ ]:
outline in pseudocode
main
    import energy_landscapes
    import propagators
    import sampling_method_1
    import sampling_method_2
    import sampling_method_...
    #import progress_coordinates #not yet written; where to draw the line between this and the sampling method is unclear
    import comparison_methods #includes plotting methods

comparison_methods.compare_something([energy_landscapes.l1, energy_landscapes.l2], 
                                     propagators.p1, 
                                     [sampling_method_1, sampling_method_2, ...], 
                                     progress_coordinate.pc,
                                     cutoff_type = 'molecular_time', 
                                     cutoff = 10ns
                                     n_replicates = 3)

def compare_something:
    for ls in args[0]:
        #landscape_ests = []
        for sm in args[2]:
            sm_ests = []
            for r in range(n_replicates):
                sm_ests.append(sm(ls, p1, pc, cutoff_type, cutoff))
            #landscape_ests.append(sm_ests)
            
            comparison_methods.numerical_comparison(ls, sm_ests, other_ground_truths (i.e. kinetics from long simulations))
            comparison_methods.plotting_methods(ls, sm_ests, other_ground_truths (i.e. kinetics from long simulations))
                    

In [ ]:
#write a wrapper for each sampling method to return standardized things and then a bootstrapping method that takes the wrappers as arguments
#feed the output of bootstrapping into an analysis method for plotting several landscape estimates with error bars on one plot
#    then replace the contents of testing_v2 with the above

#after that generalize to 2d landscapes. Hold off on 3d for the moment for ease of visualization
#    review any TODO statements

In [ ]:
ting several landscape estimates with error bars on one plot
    then replace the contents of testing_v2 with the above

after that generalize to 2d landscapes. Hold off on 3d for the moment for ease of visualization
    review any TODO statements

In [ ]:
What do we want to be able to compute for each method?:
                                     
1. energy landscape as a function of some progress coordinate vs real landscape based on PMF only: 
    inputs: trajectory, 
            weights if applicable, 
            progress coordinate, 
            real landscape

2. energy landscape as a function of some progress coordinate vs real landscape based on MSM: 
    inputs: trajectory, 
            MSM construction parameters(lag time, binning algorithm and its parameters, etc),
            weights if applicable, 
            progress coordinate, 
            real landscape

3. MFPT between arbitrary macrostates based on complete transitions and weights only
    inputs: trajectory,
            weights if applicable, 
            macrostate classifier
            
4. MFPT between arbitrary macrostates based on MSM
    inputs: trajectory,
            MSM construction parameters(lag time, binning algorithm and its parameters, etc),
            weights if applicable, 
            macrostate_classifier,
            progress coordinate if used for binning


In [ ]:
import propagators
import energy_landscapes
import long_simulation


In [ ]:
kT = 1
x_init_coord = 0
dt = 0.001
nsteps = 1000
save_period = 100
n_parallel = 4
nbins = 10
system1 = energy_landscapes.unit_double_well()

landscape = long_simulation.recover_energy_landscape(propagators.propagate_nd, system1, kT, x_init_coord, dt, nsteps, save_period, n_parallel, nbins)


In [ ]:
#run dynamics and recover the landscape
#steps per round
nsteps = 1000
#number of rounds
nrounds = 100000

nbins = 101
x_init_coord = -8
n_parallel = 1

#note that e_data here refers to energies, not ensembles
x_data, e_data, recorded_positions = long_simulation.recover_energy_landscape(fewwell_force, D, kT, dt, nsteps, nrounds, nbins, x_init_coord, n_parallel)